# Projeto 12: Validação Cruzada - Rede Neural Convolucional MNIST

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 2.7 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.model_selection import cross_val_score
import torchvision.transforms as transforms
from skorch import NeuralNetClassifier
torch.__version__

'2.0.1+cu118'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
transform = transforms.ToTensor()

In [5]:
train = datasets.MNIST(root = '.', train = True, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 102974918.00it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 102919026.19it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 125458347.33it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17366024.40it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## Etapa 3: Construção do modelo

In [6]:
class classificador_torch(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
    self.conv2 = nn.Conv2d(32, 32, (3, 3))
    self.activation = nn.ReLU()
    self.bnorm = nn.BatchNorm2d(num_features=32)
    self.pool = nn.MaxPool2d(kernel_size = (2,2))
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(in_features=32*5*5, out_features=128)
    self.linear2 = nn.Linear(128, 128)
    self.output = nn.Linear(128, 10)
    self.dropout = nn.Dropout(p = 0.2)

  def forward(self, X):
    X = X.view(-1, 1, 28, 28) ##### Tem que transformar as imagens no formato que as CNNs operam no PyTorch: (batch_size, n_dimensions, width, height)
    X = self.pool(self.bnorm(self.activation(self.conv1(X))))
    X = self.pool(self.bnorm(self.activation(self.conv2(X))))
    X = self.flatten(X)

    X = self.dropout(self.activation(self.linear1(X)))
    X = self.dropout(self.activation(self.linear2(X)))
    X = self.output(X)

    return X

In [7]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 3,
                                            batch_size = 10,
                                            train_split = False)

## Etapa 4: Validação cruzada

In [8]:
##### Primeiro temos que mudar os tipos dos dados de X e y para corresponderem aos tipos dos pesos usados na rede.
X = train.data.type('torch.FloatTensor')
y = train.targets.type('torch.LongTensor')

In [9]:
resultados = cross_val_score(classificador_sklearn, X, y, cv = 5, scoring = 'accuracy')

  epoch    train_loss      dur
-------  ------------  -------
      1        0.4433  48.8403
      2        0.3678  42.2029
      3        0.4148  41.7659
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4084  39.8715
      2        0.3713  42.6338
      3        0.3678  41.9681
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4093  39.0529
      2        0.3601  41.3225
      3        0.3939  42.4712
  epoch    train_loss      dur
-------  ------------  -------
      1        0.3914  39.5673
      2        0.3681  41.0750
      3        0.3443  42.3720
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4096  39.2389
      2        0.3676  40.8121
      3        0.3977  41.1523


In [10]:
media = resultados.mean()
desvio = resultados.std()

In [11]:
media, desvio

(0.6026999999999999, 0.20298228302544588)

In [12]:
resultados

array([0.646     , 0.20158333, 0.71783333, 0.70766667, 0.74041667])